In [1]:
import glob
import os
from pydub import AudioSegment

In [2]:
import pandas as pd
import librosa
import librosa.display

In [3]:
import numpy as np

def get_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = get_mfcc(audio, sample_rate)
        flux = get_spectral_features(audio, sample_rate)[2]
#         zero_crossing_rate = get_zero_crossing_rate(audio)
#         spectral_features = get_spectral_features(audio, sample_rate)
        chroma_features = get_chroma_features(audio, sample_rate)
        return [mfccs, flux] + chroma_features
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None

In [4]:
def get_all_features(folder):
    folders = os.listdir(folder)
    folders.sort()
    all_features = []
    for label, sub_dir in enumerate(folders): #label {0; 4}
        for file_name in glob.glob(os.getcwd() + '/' + folder +'/' + sub_dir + '/*.wav'):
#             print("Extracting file ", file_name)
            try:
                features = get_features(file_name)
            except Exception as e:
                print("Extraction error")
                continue
            all_features.append(features +  [label + 1])
    data = pd.DataFrame(all_features, columns=['mfccs', 'flux', 'chroma_features1', 'chroma_features2', 'chroma_features3', 'class_label'])
    return data

In [5]:
def get_mfcc(audio, sr):
    #Mel Frequency Cepstral Coefficient
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=100)
    mfccs_scaled = np.mean(mfccs.T,axis=0)
#     mfccs - (40, სხვადასხვა), mfccsscaled - (40,)
    return mfccs_scaled

In [6]:
def get_zero_crossing_rate(audio):
    #ZERO_CROSSING_RATE
    rate = librosa.feature.zero_crossing_rate(audio)
    rate_scaled = np.mean(rate.T,axis=0)
#     rate - (1, სხვადასხვა)
    return rate_scaled

In [7]:
def get_spectral_features(audio, sr):
    sp = librosa.feature.spectral_centroid(audio)
    S, phase = librosa.magphase(librosa.stft(audio))
    a = librosa.feature.spectral_rolloff(S=S, sr=sr)
    onset_env = librosa.onset.onset_strength(y=audio, sr=sr)
    onset_env.shape = (onset_env.shape[0], 1)
    sp_scaled = np.mean(sp.T,axis=0)
    a_scaled = np.mean(a.T,axis=0)
    onset_env_scaled = np.mean(onset_env,axis=0)
    return [sp_scaled, a_scaled, onset_env_scaled]

In [8]:
def get_chroma_features(audio, sr):    
    # Chroma features (needs scaling)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr) #N1
    chroma_cens = librosa.feature.chroma_cens(y=audio, sr=sr) #Chroma energy normalized statistics
    chroma_cq = librosa.feature.chroma_cqt(y=audio, sr=sr)
#     samive - (12, სხვადასხვა)
    chroma_stft_scaled = np.mean(chroma_stft.T,axis=0)
    chroma_cens_scaled = np.mean(chroma_cens.T,axis=0)
    chroma_cq_scaled = np.mean(chroma_cq.T,axis=0)
    return [chroma_stft_scaled, chroma_cens_scaled, chroma_cq_scaled]

In [9]:
def get_pitch(audio, sr):
    # Pitch (needs scaling)
    pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
#     pitches - (1025, სხვადასხვა)
    pitches_scaled = np.mean(pitches.T,axis=0)
    return pitches_scaled

In [10]:
data = get_all_features('data')

In [186]:
data = data.sample(frac=1).reset_index(drop=True)
data

,mfccs,flux,chroma_features1,chroma_features2,chroma_features3,class_label
0,"[-580.9515, 59.095158, 4.1009164, 17.406452, -...",[1.4029855],"[0.4783491, 0.41302508, 0.36737356, 0.2657181,...","[0.31292143765471175, 0.3156018835981847, 0.31...","[0.5226455, 0.53991604, 0.52330285, 0.46647242...",2
1,"[-451.09662, 120.30325, 35.49298, 27.94192, 22...",[1.3774267],"[0.5897228, 0.5362806, 0.5104867, 0.5251799, 0...","[0.3566469790244338, 0.3551025436576043, 0.317...","[0.71202505, 0.69777024, 0.6863941, 0.64191115...",2
2,"[-490.932, 44.57608, -4.013698, 7.2939653, -16...",[1.8068326],"[0.59243655, 0.5102444, 0.37807012, 0.21584116...","[0.3704449922082988, 0.3610205632789337, 0.302...","[0.61791337, 0.61941004, 0.53759724, 0.4715223...",2
3,"[-537.86285, 119.50876, 4.1578264, 9.344819, 1...",[1.0708644],"[0.6031261, 0.4787752, 0.46433723, 0.49029496,...","[0.4071495201839487, 0.32411929331257466, 0.27...","[0.7551209, 0.63391703, 0.5562558, 0.5182386, ...",2
4,"[-599.2242, 55.74967, -2.06318, 24.055681, 7.0...",[1.1338803],"[0.46802458, 0.415302, 0.42737782, 0.47259587,...","[0.2930406054584925, 0.2516519532285148, 0.223...","[0.6212043, 0.58510137, 0.5543627, 0.5893392, ...",1
...,...,...,...,...,...,...
928,"[-503.05307, 68.65405, 13.616894, 29.99, 11.72...",[1.1210471],"[0.57436705, 0.69312066, 0.6966968, 0.46135807...","[0.3326041837222495, 0.4093088447542526, 0.382...","[0.58311737, 0.7337678, 0.6397954, 0.47961947,...",3
929,"[-360.51633, 74.003845, 1.694712, 58.93895, -5...",[1.6287421],"[0.45031384, 0.6748576, 0.50095654, 0.5209981,...","[0.20517272589192911, 0.23171906380919421, 0.2...","[0.5192524, 0.55190384, 0.54977286, 0.55118847...",1
930,"[-353.77402, 103.14397, 11.051961, 36.468407, ...",[1.1955951],"[0.615571, 0.5945049, 0.6238268, 0.6536198, 0....","[0.28395209507001806, 0.30842738218477644, 0.3...","[0.67426586, 0.7230175, 0.70601594, 0.5835953,...",5
931,"[-526.89197, 66.149765, 13.814211, 7.6289062, ...",[1.2127483],"[0.6235193, 0.5185574, 0.51093954, 0.48346716,...","[0.2991558321737164, 0.2579775063116982, 0.273...","[0.6786987, 0.6314706, 0.64556706, 0.6358603, ...",2


In [187]:
test_data = get_all_features('test')
test_data = test_data.sample(frac=1).reset_index(drop=True)
test_data

,mfccs,flux,chroma_features1,chroma_features2,chroma_features3,class_label
0,"[-506.1684, 65.66203, 33.701477, 43.65632, -4....",[1.5657881],"[0.40114653, 0.3731631, 0.48328295, 0.48039213...","[0.2920471097086192, 0.2930405019111327, 0.290...","[0.6231679, 0.6254522, 0.6280429, 0.5692649, 0...",1
1,"[-485.59055, 63.222195, 15.867859, 20.553497, ...",[1.5205839],"[0.6946735, 0.6309911, 0.6145359, 0.60764027, ...","[0.3931982878551257, 0.36561492294997605, 0.28...","[0.8065624, 0.7632137, 0.6045693, 0.60074776, ...",4
2,"[-342.99542, 101.27446, 11.277179, 30.345806, ...",[1.2384385],"[0.58700377, 0.59619796, 0.64292204, 0.6882647...","[0.3315824026474964, 0.34064613777480524, 0.32...","[0.707168, 0.74272794, 0.6717075, 0.63860255, ...",3
3,"[-444.93436, 50.723774, 18.369139, 16.19961, 7...",[1.4494555],"[0.59621143, 0.494852, 0.43561548, 0.47940192,...","[0.34556402520548585, 0.25663220899678424, 0.2...","[0.7199497, 0.60318553, 0.60958517, 0.5880586,...",3
4,"[-555.39404, 77.74877, 32.77244, 44.852264, -1...",[1.6584789],"[0.44886804, 0.37058583, 0.36358932, 0.3548484...","[0.32936709750906745, 0.27402799280481394, 0.2...","[0.6527068, 0.6430368, 0.6005346, 0.55036426, ...",1
5,"[-523.36505, 68.824005, 23.7662, 25.582096, 1....",[1.4281317],"[0.5754428, 0.48190477, 0.5054778, 0.5513183, ...","[0.23129685834179017, 0.23201293380356736, 0.2...","[0.563202, 0.54613763, 0.5953624, 0.62446225, ...",5
6,"[-507.55008, 103.88616, 29.129068, 21.950869, ...",[1.6406921],"[0.250677, 0.2362923, 0.25260675, 0.24584621, ...","[0.19641716906263423, 0.17706025103407974, 0.1...","[0.43719903, 0.41463712, 0.41929033, 0.4078852...",2
7,"[-429.3955, 84.80725, 18.176964, 26.201424, -7...",[1.9984628],"[0.4646005, 0.44519487, 0.4020764, 0.37762818,...","[0.29015640189163744, 0.28485227585615125, 0.2...","[0.5988225, 0.59547395, 0.5745526, 0.5712071, ...",4
8,"[-473.3743, 66.0062, 31.40291, 21.405191, 2.69...",[1.5360807],"[0.7186288, 0.74076194, 0.6371092, 0.60031646,...","[0.3600979438466503, 0.35802463179033467, 0.27...","[0.7621373, 0.7379337, 0.6045187, 0.5916448, 0...",5
9,"[-475.99872, 97.237366, 26.406332, 29.371502, ...",[1.9520866],"[0.51993966, 0.5636841, 0.56326824, 0.4756088,...","[0.31744241277932217, 0.34654344375363483, 0.3...","[0.70439345, 0.69583005, 0.72975284, 0.6737864...",5


In [189]:
from sklearn.model_selection import train_test_split 
labels = [[i] for i in data['class_label']]
testLabels = [[i] for i in test_data['class_label']]
y_train = np.array(labels)
y_test = np.array(testLabels)

# x_train, x_test, y_train, y_test = train_test_split(data, np.array(labels), test_size=0.1, random_state = 127)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(839, 6) (94, 6) (933, 1) (25, 1)


# One-hot encode

In [190]:
from sklearn.preprocessing import OneHotEncoder
def get_one_hot(y):
    encoder = OneHotEncoder(sparse=False)
    y_onehot = encoder.fit_transform(y)
    return y_onehot

In [191]:
y_onehot_train = get_one_hot(y_train)
y_onehot_test = get_one_hot(y_test)
print(y_onehot_train.shape, y_onehot_test.shape)

(933, 5) (25, 5)


/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/mariam/anaconda3/envs/ml/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "ca

# აქ გვინდა რომ ყველა ფიჩერზე გვქონდეს წვდომა და მაგიტომ ამოღებისას უნდა შევცვალოთ როგორცაა შენახული დანარჩენი არაფრის შეცვლა არ მოგვიწევს

In [192]:
def get_processed_X(x):
    X = x.drop(columns=['class_label'])
    X = X.values
    ls = [] #featurebi gvinda da magitom
    for i in range(X.shape[0]):
        features = []
        for j in range(X.shape[1]):
            for k in X[i][j]:
                features.append(k)
        ls.append(features)
    res = np.array(ls)
    return res

In [193]:
ls_train = get_processed_X(data)
ls_test = get_processed_X(test_data)
print(ls_train.shape, ls_test.shape)

(933, 137) (25, 137)


 # ნეირონული ქსელის კოდი

In [194]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [195]:
import math

def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]

In [196]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = []
    for i in range(z3.shape[0]):
        z = [z3[i, j] for j in range(z3.shape[1])]
        h.append(softmax(z))
    h = np.array(h)
#     h = sigmoid(z3)
    return a1, z2, a2, z3, h

In [197]:
def cost(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

#     J = (-1 / m) * np.sum(y * np.log(h).T) #softmax
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    
    # add the cost regularization term
    J += (float(learning_rate) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))
    
    return J

In [198]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [199]:
def backprop(params, input_size, hidden_size, num_labels, X, y, learning_rate):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)
    
    J = cost(params,input_size, hidden_size, num_labels, X, y, learning_rate)

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)
        
        d3t = ht - yt  # (1, 10)
        
        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * learning_rate) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * learning_rate) / m
    
    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))
    
    return J, grad

In [200]:
# initial setup
input_size = ls_train.shape[1]
hidden_size = 50
num_labels = y_onehot_train.shape[1]
learning_rate = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = ls_train.shape[0]
ls_train = np.matrix(ls_train)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape 

((50, 138), (5, 51))

In [201]:
a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape
# print(h)
print(cost(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate))

2.566509344509288


In [202]:
J, grad = backprop(params, input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate)
J, grad.shape

(2.566509344509288, (7155,))

In [203]:
def get_accuracy(h, y):
    sm = 0
    for a,i in zip(h, y):
        sm+=a[(i[0]-1)]
    sm/= y.shape[0]
    return sm

In [204]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=backprop, x0=params, args=(input_size, hidden_size, num_labels, ls_train, y_onehot_train, learning_rate), 
                method='TNC', jac=True, options={'maxiter': 250})
fmin

     fun: 1.8980075388243827
     jac: array([ 1.65020719e-10,  5.08136107e-05, -3.10689467e-05, ...,
       -2.40099996e-04,  5.14158134e-05, -2.80029817e-03])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 33
  status: 3
 success: False
       x: array([-0.10241274,  0.0471971 , -0.02898904, ..., -0.02435867,
        0.04797728,  0.68066681])

In [205]:
ls_train = np.matrix(ls_train)
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(ls_train, theta1, theta2)
# y_pred_train = np.array(np.argmax(h, axis=1) + 1)

accuracy = get_accuracy(h, y_train)
print(h)
print(accuracy)


[[0.09510809 0.25668782 0.1228489  0.1826161  0.34273909]
 [0.04166102 0.45163315 0.0137757  0.15081846 0.34211167]
 [0.00686322 0.07620514 0.77921654 0.08379175 0.05392336]
 ...
 [0.16602049 0.1049376  0.00857759 0.20563315 0.51483116]
 [0.0817321  0.28370734 0.04826139 0.20456334 0.38173583]
 [0.10374183 0.09508668 0.01270601 0.33366113 0.45480435]]
0.4063553320875583


In [206]:
ls_test = np.matrix(ls_test)

a1, z2, a2, z3, h = forward_propagate(ls_test, theta1, theta2)
# y_pred = np.array(np.argmax(h, axis=1) + 1)

accuracy = get_accuracy(h, y_test)
print(accuracy)

0.3141252830851655


In [163]:
# correct = [1 if a == b else 0 for (a, b) in zip(y_pred_train, y_train)]
# accuracy = (sum(map(int, correct)) / float(len(correct)))
# print(accuracy)

In [164]:
# correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y_test)]
# accuracy = (sum(map(int, correct)) / float(len(correct)))
# print(accuracy)

# ჩაწერა-წაკითხვა

In [127]:
# thetas = []
# thetas.append(theta1)
# thetas.append(theta2)
# np.save('weights', thetas)
# theta1, theta2 = np.load('weights.npy', allow_pickle = True)


# გვჭირდება !pip install XlsxWriter

In [162]:
import xlsxwriter
workbook = xlsxwriter.Workbook("result.xlsx")
worksheet = workbook.add_worksheet()
row = 0 
col = 0

for a, b, c, d, e in h:
    worksheet.write(row, col, a)
    worksheet.write(row, col+1, a)
    worksheet.write(row, col+2, a)
    worksheet.write(row, col+3, a)
    worksheet.write(row, col+4, a)
    row+=1
    
workbook.close()